# 運動のシミュレーション

ニュートン力学によると、質点の運動は

$$
m \frac{d^2 \vec{r}}{dt^2} = \vec{F} ,
\tag{1}
$$

のような常微分方程式（ordinary differential equation）に支配されている[<sup id="cite_ref-1">[1]</sup>](#cite_note-1)[<sup id="cite_ref-2">[2]</sup>](#cite_note-2)。ここで $m$ は質点の質量、$\vec{r}$ は質点の位置、$\vec{F}$ は質点に働く力、$t$ は時間である。もし力 $\vec{F}$ が既知であり、初期条件として時刻 $t = t_0$ での質点の位置と速度が与えられたならば、その後の質点の運動は常微分方程式 (1) の初期値問題（initial value problem）を解くことによって完全に決定される。

今回は常微分方程式の初期値問題の簡単な数値解法を学び、質点の運動のシミュレーションを行ってみよう。

**注意**：実際の研究では、よく整備されたライブラリが使えるのなら、それを利用すべきである。今回の場合で言うと、例えば [SciPy](https://www.scipy.org/scipylib/) というライブラリに [scipy.integrate.solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) という関数がある。また、今回主に扱うオイラー法は、数学的にもプログラム的にも簡単で教育的ではあるが、（局所）打切り誤差も大きく、理論的な安定性も芳しくないので、常微分方程式の数値解法としてはあまり実用的であるとは言えない。

## オイラー法

ここでは少し一般的に、ある $n$ 個の物理量の組をベクトル $\vec{X} = \left(X_1,\dots,X_n\right)$ のように書いて、その時間変化を考える[<sup id="cite_ref-3">[3]</sup>](#cite_note-3)[<sup id="cite_ref-4">[4]</sup>](#cite_note-4)。$\vec{X}$ の時間についての一階微分が、既知の関数 $\vec{f}(\vec{X},t)$ によって

$$
\frac{d \vec{X}(t)}{dt} = \vec{f}(\vec{X}(t),t) ,
\tag{2}
$$

のように与えられていると仮定しよう。十分に小さな時間変化 $\Delta t$ に対し、$\vec{X}(t+\Delta t)$ のテイラー展開を考えると、

$$
\begin{split}
\vec{X}(t + \Delta t)
&= \vec{X}(t) + \frac{d \vec{X}(t)}{dt} \Delta t + \mathcal{O} \left( (\Delta t)^2 \right) \\ 
&= \vec{X}(t) + \vec{f}(\vec{X}(t),t) \Delta t + \mathcal{O} \left( (\Delta t)^2 \right) ,
\end{split}
\tag{3}
$$

となる。式 (3) の右辺の剰余項 $\mathcal{O}\left((\Delta t)^2\right)$ を無視すれば、時刻 $t$ での $\vec{X}$ と $\vec{f}$ の値から、時刻 $(t + \Delta t)$ での $\vec{X}$ の値を近似的に求めることができる：

$$
\vec{X}(t + \Delta t)
\simeq
\vec{X}(t) + \vec{f}(\vec{X}(t),t) \Delta t .
\tag{4}
$$

よって、ある初期時刻 $t = t_0$ での $\vec{X}(t_0)$ の値が与えられていれば、式 (4) のステップを何回も繰り返せば、その後の $\vec{X} $の時間発展が分かることになる。式 (4) においてステップ幅 $\Delta t$ を小さくすれば近似がよくなるが[<sup id="cite_ref-5">[5]</sup>](#cite_note-5)、当然その分だけ計算に時間がかかることになる。式 (4) を用いた常微分方程式の初期値問題の近似解法をオイラー法（Euler method）と呼ぶ[<sup id="cite_ref-6">[6]</sup>](#cite_note-6)。

## 一次元調和振動子

ばね振り子のような、一次元調和振動子（one-dimensional harmonic oscillator）の運動の様子を、オイラー法を用いて数値的に計算してみよう。解くべき微分方程式は

$$
\frac{d^2 x(t)}{dt^2} = - \omega^2 x(t) ,
\tag{5}
$$

である。ここで $x$ は調和振動子の変位、$\omega$ は角振動数である[<sup id="cite_ref-7">[7]</sup>](#cite_note-7)。

式 (5) は二階の微分方程式であるが、$v_x = \frac{dx}{dt}$ を使うと、連立の一階微分方程式として書き直すことができる：

$$
\begin{align}
\frac{dx(t)}{dt} &= v_x(t) \tag{6a}, \\
\frac{dv_x(t)}{dt} &= - \omega^2 x(t) \tag{6b} .
\end{align}
$$

よって、$\vec{X} = (x, v_x)$ として式 (4) を適用すると、

$$
\begin{align}
x(t+\Delta t) &\simeq x(t) + v_x(t) \Delta t ,
\tag{7a} \\
v_x(t+\Delta t) &\simeq v_x(t) - \omega^2 x(t) \Delta t ,
\tag{7b}
\end{align}
$$

となる。時刻 $t = 0$ での初期条件 $x(0)$ と $v_x(0)$ が与えられれば、式 (7) のステップを繰り返すことにより、系の時間発展を近似的に求めることができる[<sup id="cite_ref-8">[8]</sup>](#cite_note-8)。

In [ ]:
# オイラー法による調和振動子のシミュレーション

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# 最終時刻
t_max = 2

# ステップ幅
dt = 0.00001

# 初期条件と角振動数
x0 = 1
vx0 = 0
omega = 4 * np.pi

# 初期化
t = 0
x = x0
vx = vx0

# グラフ用のリスト
t_points = [t]
x_points = [x]
vx_points = [vx]

while t < t_max:
    # オイラー法で時間をdt進める
    x_new = x + vx * dt
    vx_new = vx - omega ** 2 * x * dt
    
    t = t + dt
    x = x_new
    vx = vx_new
    
    # グラフ用に点を追加
    t_points.append(t)
    x_points.append(x)
    vx_points.append(vx)
    
# グラフを描く
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)

ax1.plot(t_points, x_points, label='$x$ (Euler)')
ax2.plot(t_points, vx_points, label='$v_x$ (Euler)')

ax1.set_ylabel('$x$')
ax1.grid()
ax1.legend()

ax2.set_xlabel('$t$')
ax2.set_ylabel('$v_x$')
ax2.grid()
ax2.legend()

plt.show()

ここでは [subplots 関数](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html)にオプション引数を渡して、横軸を共有した [Axes オブジェクト](https://matplotlib.org/api/axes_api.html#the-axes-class)を二つ（`ax1` と `ax2`）作り、それぞれに $x$ と $v_x$ の時間変化を描いている。

単位系はどのようにとってもよいが、時間の単位を秒、長さの単位を cm にとって、ばねに吊るされた重りを、ばねの自然長からいくらか伸ばしたところで静かに手を離した状況を考えると、この結果の物理的な意味がイメージがしやすいだろう。

## 練習問題

(1) この調和振動子の運動の解析解は

$$
\begin{align}
x(t) &= x(0) \cos(\omega t) + \frac{v_x(0)}{\omega} \sin(\omega t) , \tag{8a} \\
v_x(t) &= v_x(0) \cos(\omega t) -x(0) \, \omega \sin(\omega t) , \tag{8a} \\
\end{align}
$$

で与えられる。これらの曲線をグラフに（例えば点線で）描き入れ、数値解と比較せよ。

(2) 初期条件 $x(0)$、$v_x(0)$ や角振動数 $\omega$ を少し変えてみて、運動がどのように変化するか観察せよ。

(3) ステップ幅 $\Delta t$ を大きくしてみよ。例えば、$\Delta t = 0.01$ として、どのような結果が得られるか確認せよ。また、そのような結果が得られた理由を考察せよ。

## 斜方投射

ボールを斜め上の方向に投げるときのことを考えよう。ボールは質量 $m$ の質点であるとする（つまり回転などは考えない）。水平方向に $x$ 軸、鉛直上向きに $y$ 軸をとる。速度の二乗に比例した空気抵抗を仮定すると[<sup id="cite_ref-9">[9]</sup>](#cite_note-9)、解くべき微分方程式は

$$
\begin{align}
\frac{d x(t)}{dt} &= v_x(t) , \tag{9a} \\
\frac{d y(t)}{dt} &= v_y(t) , \tag{9b} \\
\frac{d v_x(t)}{dt} &= - \frac{b}{m} v(t) v_x(t) , \tag{9c} \\
\frac{d v_y(t)}{dt} &= - \frac{b}{m} v(t) v_y(t) - g, \tag{9c} \\
\end{align}
$$

である。ただし、$v(t) = \sqrt{[v_x(t)]^2 + [v_y(t)]^2}$ であり、$b$ は空気抵抗係数、$g$ は重力加速度である。時刻 $t = 0$ に初速度 $v_0$ で水平方向から角度 $\theta$ だけ上の向きにボールを投げたとしよう。MKS単位系で適当なパラメーター[<sup id="cite_ref-10">[10]</sup>](#cite_note-10)をとったシミュレーションは次のようになる。

In [ ]:
# オイラー法による斜方投射のシミュレーション

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# ステップ幅
dt = 0.00001

# パラメーター
m = 0.2
g = 9.8
b = 0.002
v0 = 20
y0 = 1.7  # ボールを離す高さ（人間の身長程度）
theta = np.radians(45)

# 初期化
t = 0
x = 0
y = y0
vx = v0 * np.cos(theta)
vy = v0 * np.sin(theta)

# グラフ用のリスト
x_points = [x]
y_points = [y]

# 地面に接触するまでループする
while y >= 0:
    # オイラー法で時間をdt進める
    v = np.sqrt(vx ** 2 + vy ** 2)
    
    x_new = x + vx * dt
    y_new = y + vy * dt
    vx_new = vx - b / m * v * vx * dt
    vy_new = vy - (b / m * v * vy + g) * dt
    
    t = t + dt
    x = x_new
    y = y_new
    vx = vx_new
    vy = vy_new
    
    # グラフ用に点を追加
    x_points.append(x)
    y_points.append(y)

# 接地した時の時刻と座標を表示
print(f't = {t}, y = {y}, x = {x}')

# グラフを描く
fig, ax = plt.subplots()
ax.plot(x_points, y_points)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_aspect('equal')
ax.grid()
plt.show()

## 練習問題

(4) 空気抵抗がなく（$b=0$ [kg/m]）地面の高さ（$y(0)=0$ [m]）からボールを投げた場合、同じ初速度でもっともボールを遠くまで投げることができる角度は 45°である。さて、このシミュレーションのように、空気抵抗があり、ボールを投げる高さもある場合、もっともボールを遠くまで投げることのできる角度はどうなるだろうか？次の中から選べ。

(a) 45°より小さな角度。

(b) 45°のまま変わらない。

(c) 45°より大きな角度。

(5) 月面（$g = 1.6$ [m/s$^2$]、$b = 0$ [kg/m]）でボール投げをすると、地球の場合と同じ初速度、同じ角度でどのくらい遠くまでボールを投げることができるだろうか？

## 演習課題

太陽の周りを動く一つの天体（惑星や彗星など）の運動を考える。天体は太陽に比べて非常に軽いとし、太陽も天体も質点として議論することにすると[<sup id="cite_ref-11">[11]</sup>](#cite_note-11)、運動方程式は次のようになる：

$$
m \frac{d^2 \vec{r}}{dt^2} = - \frac{GMm}{|\vec{r}|^2} \frac{\vec{r}}{|\vec{r}|} .
\tag{10}
$$

ここで $\vec{r}$ は太陽を原点としたときの天体の位置、$t$ は時間、$M$ と $m$ はそれぞれ太陽と天体の質量、$G$ は万有引力定数である。中心力のみが働いているので、この運動は平面運動となる。その平面内に二次元直交座標系（$x$ 軸と $y$ 軸）をとると、式 (10) は次のような連立微分方程式として書き下せる：

$$
\begin{align}
\frac{d x(t)}{dt} &= v_x(t) , \tag{11a} \\
\frac{d y(t)}{dt} &= v_y(t) , \tag{11b} \\
\frac{d v_x(t)}{dt} &= - \frac{GM}{[r(t)]^3} x(t) , \tag{11c} \\
\frac{d v_y(t)}{dt} &= - \frac{GM}{[r(t)]^3} y(t) . \tag{11c} \\
\end{align}
$$

ただし $r(t) = \sqrt{[x(t)]^2 + [y(t)]^2}$ である。長さの単位は au (天文単位；astronomical unit；1 au は、ほぼ地球と太陽との平均距離に等しい)、時間の単位は年をとることにする。このとき、万有引力定数と太陽質量の積は、

$$
GM \simeq 4 \pi^2 \text{[au$^3$/yr$^2$]} ,
\tag{12}
$$

となる[<sup id="cite_ref-12">[12]</sup>](#cite_note-12)。

連立微分方程式 (11) をもとに、オイラー法を用いて、天体の運動をシミュレーションしてみよう。平面上の運動では $t = 0$ での初期条件として四つのパラメーターが必要であるが、楕円軌道を考えて、「天体の初期位置は太陽からもっとも遠い軌道上の点にとる[<sup id="cite_ref-13">[13]</sup>](#cite_note-13)、移動方向は半時計周りとする」という条件を課す。さらに、天体の初期位置を $x$ 軸の正の方向とすると、残りの自由なパラメーターは二つである。これらを楕円軌道[<sup id="cite_ref-14">[14]</sup>](#cite_note-14)の軌道長半径 $a$ と離心率 $e$ で決めることにすると、次のような式が得られる：

$$
\begin{align}
x(0) &= r_0 , \tag{13a} \\
y(0) &= 0 , \tag{13b} \\
v_x(0) &= 0, \tag{13c} \\
v_y(0) &= r_0 \omega_0 . \tag{13d}
\end{align}
$$

ただし、

$$
r_0 = a (1 + e) , \qquad
\omega_0 = \frac{\sqrt{GMa(1-e^2)}}{r_0^2} , \tag{14}
$$

である[<sup id="cite_ref-15">[15]</sup>](#cite_note-15)。

(1) 天体の運動のシミュレーションを行うプログラムを作成せよ。まず、地球（公転周期 1.00001742096 [yr]）の軌道のパラメーター：

$$
a = 1.000001018 \text{ [au]}, \qquad
e = 0.0167086 ,
\tag{15}
$$

を用い、$\Delta t = 0.00001$ [yr] として、$t_\text{max} = 3$ [yr] ほど計算する。$x$-$y$ 平面での軌道をグラフに描くと、ほぼ円軌道になっているだろう[<sup id="cite_ref-16">[16]</sup>](#cite_note-16)[<sup id="cite_ref-17">[17]</sup>](#cite_note-17)。また、$x$ 座標（もしくは $y$ 座標でもよいが）を時間の関数として描くと、公転周期がほぼ一年になっているだろう。それらを確認したら、提出する際には次のパラメーターで表される架空の天体（公転周期約 2 [yr]、近日点距離 0.8 [au]、遠日点距離 2.4 [au]）の運動を少なくとも $t_\text{max} = 3$ [yr] 以上まで計算し、$x$-$y$ 平面での軌道をグラフに描け。

$$
a = 1.6 \text{ [au]} , \qquad
e = 0.5 .
\tag{16}
$$

In [ ]:
# 課題解答14.1  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np




## 発展課題

余裕があればやってみてください。

(2) ハレー彗星（公転周期 75.32 [yr]、近日点距離 0.586 [au]、遠日点距離 35.082 [au]）の軌道

$$
a = 17.834 \text{ [au]} , \qquad
e = 0.96714 .
\tag{17}
$$

についてシミュレーションを行ってみよう。演習課題 (1) で作ったプログラムにおいて同じステップ幅を使って 80 [yr] ほど計算すると、残念ながらオイラー法の誤差によって楕円軌道から少しずれてしまう（元の位置に戻って来ない）。ステップ幅を小さくしたり、ルンゲ＝クッタ法（Runge–Kutta method）のような高次の近似法を用いたりしてもよいが、ここでは以下のことを考えよう。

いま、保存力のみを考えているので、運動方程式は

$$
\begin{align}
\frac{d\vec{r}(t)}{dt} &= \vec{v}(t) , \tag{18a} \\
\frac{d\vec{v}(t)}{dt} &= \vec{f}\left(\vec{r}(t)\right) , \tag{18b}
\end{align}
$$

のような形で与えられている。これを

$$
\begin{align}
\vec{r}(t+\Delta t) &\simeq \vec{r}(t) + \vec{v}(t) \Delta t, \tag{19a} \\
\vec{v}(t+\Delta t) &\simeq \vec{v}(t) + \vec{f}\left(\vec{r}(t)\right) \Delta t , \tag{19b}
\end{align}
$$

と近似するのがオイラー法であった。式 (19) の写像は力学的エネルギーを保存せず、繰り返し適用することによって力学的エネルギーが変化してしまう。実は、式 (19) の代わりに

$$
\begin{align}
\vec{r}(t+\Delta t) &\simeq \vec{r}(t) + \vec{v}(t) \Delta t, \tag{20a} \\
\vec{v}(t+\Delta t) &\simeq \vec{v}(t) + \vec{f}\left(\vec{r}(t+\Delta t)\right) \Delta t , \tag{20b}
\end{align}
$$

を使うと、力学的エネルギー保存に対する誤差が改善することが知られている（式 (20b) の右辺第二項に注意）。この近似法はシンプレクティック・オイラー法（symplectic Euler method）と呼ばれている[<sup id="cite_ref-18">[18]</sup>](#cite_note-18)。これを使ってみよう。まず、演習課題 (1) のようにオイラー法で地球の軌道を計算するとき、ステップ幅を $\Delta t = 0.001$ [yr] とすると、誤差により軌道が閉じない（一周するとずれる）ことを確認する。次に、プログラムをシンプレクティック・オイラー法を使うように書き換えると、$\Delta t = 0.001$ [yr] のままでも軌道が閉じるはずである。このことが確認出来たら、ハレー彗星の軌道計算を少なくとも $t_\text{max} = 150$ [yr] 以上行い、$x$-$y$ 平面での軌道をグラフに描け。

In [ ]:
# 課題解答14.2  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np




(3) 中心力は角運動量保存則を導き、平面内での運動をもたらすが、天体の軌道が閉じた楕円を描くのは中心力が距離の二乗に反比例しているか、もしくは調和振動子の場合のみである。実際、式 (10) の代わりに

$$
m \frac{d^2 \vec{r}}{dt^2} = - \frac{GMm}{|\vec{r}|^2}
\left( 1 + \frac{C_2}{|\vec{r}|^2} \right)
\frac{\vec{r}}{|\vec{r}|} ,
\tag{21}
$$

を用いると[<sup id="cite_ref-19">[19]</sup>](#cite_note-19)、楕円軌道の長軸の向きが回転する（近日点移動）。このことをシンプレクティック・オイラー法を用いてシミュレーションしてみよう。水星（公転周期 0.240846 [yr]、近日点距離 0.307499 [au]、遠日点距離 0.466697 [au]）の軌道パラメーター：

$$
a = 0.387098 \text{ [au]} , \qquad
e = 0.205630 ,
\tag{22}
$$

を用い、$C_2 = 0.005$ [au$^2$] [<sup id="cite_ref-20">[20]</sup>](#cite_note-20)[<sup id="cite_ref-21">[21]</sup>](#cite_note-21)、$\Delta t = 0.0001$ [yr] として[<sup id="cite_ref-22">[22]</sup>](#cite_note-22)、$t_\text{max} = 1$ [yr] までシミュレーションを行い[<sup id="cite_ref-23">[23]</sup>](#cite_note-23)、$x$-$y$ 平面での軌道をグラフに描け。

In [ ]:
# 課題解答14.3  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np




## 脚注

<span id="cite_note-1">1.</span> [^](#cite_ref-1)
ガリレオ・ガリレイの有名な言葉に「宇宙は数学の言葉で書かれている（L'universo è scritto in lingua matematica.）」というものがある。私のような数学が苦手な人には残念なお知らせではあるが、数学を知らないと我々の住んでいるこの宇宙を支配している物理法則を理解することはできない。

<span id="cite_note-2">2.</span> [^](#cite_ref-2)
ところで、ここではベクトル量を矢印を上に付けることで表記する。高校まではベクトルに矢印を使っていたのが、大学の教科書や専門書では太字になっていることが多く、やっとそれに慣れた学生さんからすると、なぜ太字で表記しないのか？と思うかもしれないが、（矢印や太字よりもテンソルの添字記法をよく使う人間からしたらどっちでもええやんと思うのを別にすると）Jupyter ノートブックで太字を使うとすると、あらゆる環境の Web ブラウザー上のフォントで太字が実際に明確に太く表示されることが保証されていない（気がする）から、という立派な理由がある。もっとも、一番の理由は、 [KaTeX](https://katex.org/) とは違って [MathJax](https://www.mathjax.org/) では `\bm` コマンドがサポートされていないので `\boldsymbol` と長いコマンドを使う必要があり、 `\vec` よりも 7 文字も多くタイプしなければならないのが面倒くさいからだったりする。（`\bf` や `\mathbf` はギリシャ文字が太字にならないので使いたくない。）

<span id="cite_note-3">3.</span> [^](#cite_ref-3)
$n$ 個の場合が分かりにくければ、まずは $n=1$、つまりスカラー量の場合を考えるとよい。

<span id="cite_note-4">4.</span> [^](#cite_ref-4)
$\vec{X}$ は実ベクトル（$\vec{X}\in\mathbb{R}^n$）、$t$ は実数とする。$\vec{X}$ を複素ベクトル（$\vec{X} \in \mathbb{C}^n$）としてもよいが、それは実部と虚部を分けて考えると $\vec{X} \in \mathbb{R}^{2n}$ と等価である。

<span id="cite_note-5">5.</span> [^](#cite_ref-5)
悪条件がなければ、$\Delta t$ を小さくしていけばオイラー法の打切り誤差をいくらでも小さくできる（丸め誤差などはある）。時刻 $t=0$ から $t=T$ までを $N$ ステップに分割すると $\Delta t = T/N$ であり、オイラー法による誤差の蓄積は全体で（十分に $\Delta t$ が小さければ） $N \times \mathcal{O}\left((\Delta t)^2\right)=\mathcal{O}(\Delta t)$ と期待される。現実的には $\Delta t$ を変えながら（例えば $\Delta t$ を二倍にしたりして）二回以上同じ問題を解いてみて、数値解の軌道のずれがどのくらいになるか調べてみる必要があるだろう。

<span id="cite_note-6">6.</span> [^](#cite_ref-6)
（後退オイラー法などと区別して）前進オイラー法（forward Euler method）とも呼ぶ。

<span id="cite_note-7">7.</span> [^](#cite_ref-7)
ばね定数 $k$ のばねにつながれた質量 $m$ の質点の運動の場合（静止状態を $x=0$ とする）、$\omega = \sqrt{k / m}$ である。

<span id="cite_note-8">8.</span> [^](#cite_ref-8)
このプログラムでは簡単のために毎ステップごとにリストに点を追加しているが、グラフの描画のためであれば、そんなに細かく点を保存しておく必要はない。むしろ点が多すぎるとグラフの描画が遅くなる。例えば 100 ステップごとに1回だけ点を追加するとか、もしくは後で
```python
n = 1000  # グラフで使う点の数
d = max(len(t_points) // n, 1)
t_points = t_points[::d]
x_points = x_points[::d]
vx_points = vx_points[::d]
```
のように、グラフに描画する際に適当にダウンサンプリングしてもよいだろう。

<span id="cite_note-9">9.</span> [^](#cite_ref-9)
速度に比例する粘性抵抗は無視する。

<span id="cite_note-10">10.</span> [^](#cite_ref-10)
ソフトボール（質量約 0.2 kg）を投げるとして、半径約 0.05 m、空気密度 1.2 kg/m$^3$、抗力係数 $C_D=0.5$ とすると、$b = 0.002$ kg/m 程度（野球のボールに対する[同様の計算](http://www.physics.usyd.edu.au/~cross/TRAJECTORIES/Trajectories.html)を参考にした）。地表での重力加速度が約 9.8 m/s$^2$ なのはよく知られている。一番適当なのは初速度である（このくらい飛ぶかなぁと思って適当に決めた）。

<span id="cite_note-11">11.</span> [^](#cite_ref-11)
考えている天体は太陽以外からの重力の影響を受けない、自転や扁平率も考えない、天体の質量は太陽の質量に比べてきわめて小さいので太陽は原点にじっとしている、と仮定して問題を簡単化する。

<span id="cite_note-12">12.</span> [^](#cite_ref-12)
天体の軌道長半径を $a$、公転周期を $T$ とすると、次の関係がある：

$$
\frac{a^3}{T^2} = G \frac{M + m}{4 \pi^2} .
$$

これに地球を当てはめれば式 (12) となる。$a^3$ と $T^2$ の比が定数になるというのは、ケプラーの第三法則を表している。

<span id="cite_note-13">13.</span> [^](#cite_ref-13)
$r = r_\text{max} = a(1 + e)$、$\ddot{r}=0$ となる点なので、非常に計算がしやすい。

<span id="cite_note-14">14.</span> [^](#cite_ref-14)
楕円の中心から焦点までの距離は $a e$ であり、軌道短半径 $b$ との関係は $a^2 (1 - e^2) = b^2$ である。

<span id="cite_note-15">15.</span> [^](#cite_ref-15)
太陽（つまり座標系の原点）は楕円の中心ではなく、（遠い方の）焦点に位置する。

<span id="cite_note-16">16.</span> [^](#cite_ref-16)
グラフの縦横比を調整するために [Axes オブジェクト](https://matplotlib.org/api/axes_api.html#the-axes-class)の [set_aspect メソッド](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.set_aspect.html)に `'equal'`を渡して呼びだすこと。

<span id="cite_note-17">17.</span> [^](#cite_ref-17)
ステップ幅が大きいと、誤差のために円軌道が閉じない（一周した時に位置がずれる）。

<span id="cite_note-18">18.</span> [^](#cite_ref-18)
式 (20) の代わりに

$$
\begin{align}
\vec{r}(t+\Delta t) &\simeq \vec{r}(t) + \vec{v}(t+\Delta t) \Delta t, \\
\vec{v}(t+\Delta t) &\simeq \vec{v}(t) + \vec{f}\left(\vec{r}(t)\right) \Delta t ,
\end{align}
$$

としてもよい。

<span id="cite_note-19">19.</span> [^](#cite_ref-19)
式 (21) の $C_2 / r^2$ の補正項の意味は、調べるとさまざまな文献が見つかると思うが、例えば、次の文献を参照のこと：
- James D. Wells, *When effective theories predict: the inevitability of Mercury's anomalous perihelion precession*, In: *Effective Theories in Physics: From Planetary Orbits to Elementary Particle Masses* (Springer, 2012), [arXiv:1106.1568](https://arxiv.org/abs/1106.1568).

<span id="cite_note-20">20.</span> [^](#cite_ref-20)
軌道をグラフに描いたときに分かりやすいよう、非常に大きな値にとってある（近日点で $C_2 / r^2 \simeq 0.05$）。ちなみにシュヴァルツシルト解による補正では、

$$
C_2 = \frac{a (1-e^2) GM}{c^2} \simeq 3.7 \times 10^{-9} \text{ [au$^2$]} ,
$$

となり、一世紀あたり約 43″（秒；1″は 1°の 1 / 3600）の近日点移動を引き起こす。

<span id="cite_note-21">21.</span> [^](#cite_ref-21)
プログラムのチェックとして、一度 $C_2 = 0$ [au$^2$] で計算してみてもよいかもしれない。通常の水星の軌道が描けるはずである。

<span id="cite_note-22">22.</span> [^](#cite_ref-22)
発展課題 (2) のように $\Delta t = 0.001$ [yr] ととると、（目で見える程度に）ちょっとずれる。

<span id="cite_note-23">23.</span> [^](#cite_ref-23)
ちなみに $t_\text{max} = 10$ [yr] くらいまで計算すると、ちょっと綺麗な模様が描ける。（でも提出時には $t_\text{max} = 1$ [yr] にしておいてください。その方が成否が判断しやすいので。）